In [9]:
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
import threading
import multiprocessing
from multiprocessing import Process
import time

# 1.读取行为数据

In [10]:
print('start')
start = time.time()
# train_behavior = pd.read_csv('./data/train/behaviors.tsv',sep ='\t',header=None)
train_behavior = pd.read_csv('./data/test/testBehavior_social1.csv')
###处理 behaviors.tsv 行为数据
train_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
train_behavior.dropna(axis=0, how='any', inplace=True)
train_behavior['clickHistory'] = train_behavior['clickHistory'].apply(lambda x : eval(x))
impressionLog = train_behavior['impressionLog'].str.split(' ', expand=True).stack().reset_index(level = 1,drop = True)
key, label = impressionLog.str.split('-', 1).str
data_new = pd.DataFrame()
data_new= train_behavior[['clickHistory']].join(pd.DataFrame(key,columns=['current_news']))
data_new['label'] = label
data_new['id'] = train_behavior['id']
data_new.dropna(axis=0, how='any', inplace=True)

print(data_new.shape[0])
print('time is : ',time.time() - start)


start


KeyboardInterrupt: 

In [3]:
data_new

,clickHistory,current_news,label,id
0,"[N6188, N12021, N59213, N59927, N19508, N21622...",N34090,0,U38457
0,"[N6188, N12021, N59213, N59927, N19508, N21622...",N55355,1,U38457
0,"[N6188, N12021, N59213, N59927, N19508, N21622...",N12700,0,U38457
0,"[N6188, N12021, N59213, N59927, N19508, N21622...",N46738,0,U38457
0,"[N6188, N12021, N59213, N59927, N19508, N21622...",N14368,0,U38457
...,...,...,...,...
73151,"[N52945, N25654, N38679, N20581, N19145, N5019...",N38188,0,U27142
73151,"[N52945, N25654, N38679, N20581, N19145, N5019...",N64402,0,U27142
73151,"[N52945, N25654, N38679, N20581, N19145, N5019...",N55355,1,U27142
73151,"[N52945, N25654, N38679, N20581, N19145, N5019...",N50337,0,U27142


# 2.读取新闻数据

In [4]:
# coding=utf-8
title_len = 20
train_title = pd.read_csv('./data/train/news.tsv',sep='\t',header=None, encoding='utf-8')
test_title = pd.read_csv('./data/test/news.tsv',sep='\t',header=None, encoding='utf-8')
title = pd.concat([train_title,test_title],0)
title.columns = ['id','category','sub_category','title','abstract','url','title_entity','abstract_entity']
title = title[['id','title']]
arr = []
for index ,row in title.iterrows():
    text = row['title'].split(' ')
    text = text[:title_len]
    arr.append(text)
title['title'] = arr

# 3.读取邻居数据

In [5]:
# neighbor = pd.read_csv('./data/nearst_4.csv')
# neighbor['nearst'] = neighbor['nearst'].apply(lambda x : eval(x))


In [6]:
# train_behavior = pd.read_csv('./data/train/behaviors.tsv',sep ='\t',header=None)
# train_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
# test_behavior = pd.read_csv('./data/test/behaviors.tsv',sep ='\t',header=None)
# test_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']

# behavior = pd.concat([train_behavior,test_behavior],0)


In [7]:
# neighbor_dict = dict()
# for index,row in tqdm(neighbor.iterrows()):
#     neighbor_dict[row['user']] = row['nearst']

91935it [00:13, 6703.46it/s]


# 4.多线程生成具体行为数据

## a.生成用户对应的历史记录

## b.生成对应的用户点击新闻、候选新闻

In [8]:
print('s')

s


In [9]:
#### 

punctuation = '!,;:?"\''
def removeFormat(text):
    result = re.sub(r'[{}]+'.format(punctuation),'',text).strip().lower()
    return result
def sample_data(data,name):
    x_id = []
    x_news = []
    x_title = []
    x_title_id = []
    x_label = []
    maxCount = 15
    step = 0
    for index,row in (data.iterrows()):
        
        if(step >= 0 and step % 1000 == 0):
            print(name,':',step)
        text = []
        
        if(type( row['clickHistory']) == float): continue
        
        history = row['clickHistory']
        idxs = list(range(len(history)))
        np.random.shuffle(idxs)
        his_len = len(history)
        count = 0
        for x in idxs:
            if(count == maxCount):break
            tit = title[title['id']==history[x]]['title'].values[0]
            
            if type(tit) == float:continue
            text.extend(tit)
            
            count += 1

       
        tit = title[title['id'] == row['current_news']]['title'].values[0]
        if type(tit) == float:continue
      
        x_news.append(text)
        x_title.append(tit)
        x_label.append(int(row['label']))
        x_id.append(row['id'])
        step += 1
        
       
    all_data = pd.DataFrame()
    
    all_data['id'] = x_id
    all_data['history'] = x_news
    all_data['current'] = x_title
    all_data['label'] = x_label

    all_data['history'] = all_data['history'].apply(lambda x : [removeFormat(y) for y in x])
    all_data['current'] = all_data['current'].apply(lambda x : [removeFormat(y) for y in x])
    all_data.to_csv('./data/test/order_10/test_{0}.csv'.format(name),index=False)
    print('curernt over: ',name)
print('start')
start = time.time()
pool_thread = []
groups = 46
pool = multiprocessing.Pool(processes=groups)
data_temp = data_new.iloc[:20000]

for index in range(groups+1):
    count_data = data_temp.shape[0] // groups
    pool.apply_async(sample_data,(data_temp.iloc[index * count_data:(index+1)*count_data,:],index+1))

pool.close()
pool.join() 

print('all over!')
print('time is :',time.time() - start,' s')

start
1 2 0 ::4  300 : : 

6057 8  
10 
139:::12   14:1618 0 15110
  17  1922:: 20021:0
  : :
   :0 26 : 23 
0
:240:025 
 3129 ::035:36:  30  27  00     : 

32 33
:3438
0    ::
:37: :  
0 :0:
4428  394000:045    

 42:46 0
:::: :  0    0   041:0
:
:: 

    :000 00  0:0:0 




43

 
0
  0
  0:00


00 0 
 0


0:0


 0

curernt over:  1
47 : 0
curernt over:  3
curernt over:  47
curernt over:  2
curernt over:  4
curernt over:  5
curernt over:  6
curernt over:  7
curernt over:  8
curernt over:  9
curernt over:  10
curernt over:  12
curernt over:  11
curernt over:  23
curernt over:  31
curernt over:  15
curernt over:  36
curernt over:  14
curernt over:  30
curernt over:  29
curernt over:  16
curernt over:  33
curernt over:  27
curernt over:  40
curernt over:  17
curernt over:  35
curernt over:  38
curernt over:  34
curernt over:  18
curernt over:  32
curernt over:  24
curernt over:  13curernt over: 
 21
curernt over:  28
curernt over:  26
curernt over:  19
curernt over:  20
curernt over:  37

# 5.整合所有多线程数据 

In [10]:

data = pd.DataFrame()
for index in range(1,48):
    print(index)
    now = pd.read_csv('./data/test/order_10/test_{0}.csv'.format(index))
    data = pd.concat([data,now],axis=0)
data.to_csv('./data/test/order_10/all_10_social1.csv',index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [11]:
data

,id,history,current,label
0,U38457,"['1000-hp', 'hennessey', 'maximus', 'gladiator...","['after', 'saugus', 'high', 'school', 'shootin...",0
1,U38457,"['buckeyes', 'chase', 'young', 'borrowed', 'mo...","['then', 'and', 'now', 'what', 'all', 'your', ...",1
2,U38457,"['the', '1000-horsepower', 'club', 'toyota', '...","['former', 'officer', 'held', 'up', 'hotel', '...",0
3,U38457,"['see', 'vin', 'diesels', '21st', 'birthday', ...","['5', 'arrested', 'in', 'connection', 'with', ...",0
4,U38457,"['former', 'nba', 'first-round', 'pick', 'jim'...","['13', 'popular', 'foods', 'people', 'hated', ...",0
...,...,...,...,...
31,U21920,"['buckeyes', 'chase', 'young', 'borrowed', 'mo...","['bill', 'russell', 'finally', 'accepts', 'bas...",0
32,U21920,"['lebron', 'posts', 'latest', 'triple-double',...","['halle', 'berry', 'shares', 'photo', 'of', '6...",0
33,U21920,"['newark', 'liberty', 'airports', 'terminal', ...","['miami', 'community', 'police', 'benevolent',...",0
34,U21920,"['la', 'rivalry', 'takes', 'on', 'new', 'meani...","['pair', 'of', 'new', '1987', 'buick', 'grand'...",0


In [12]:
user_news

NameError: name 'user_news' is not defined

# 6.训练数据采样

In [11]:
k = 1 ##邻居数量
def remove_sep(data):
    temp = []
    for x in data:
        if x != '[sep]':
            if(x == '[pad]'):
                x = '[PAD]'
            temp.append(x)
    return temp

def sample_neg(data,number):
    k = 1
    shape = data.shape[0]
    index = 0
    index_arr = []
    index_neighbor = []
    while index < shape:
        flag = True
        for his in range(k):
            if data.iloc[index+his]['label'] != 0:
                flag = False
                break
        if(flag):
            for pos in range(k):
#                 index_neighbor.append(neighbor[neighbor['user'] == data.iloc[pos+index]['id']]['nearst'].values)
                index_arr.append(pos + index)
            index += k
            if(index + k >= shape):break
            while(data.iloc[index]['label'] != 1):
                index += 1
                if(index + k >= shape):break
            if(index + k >= shape):break
            index_arr.append(index)
#             index_neighbor.append(neighbor[neighbor['user'] == data.iloc[index]['id']]['nearst'].values)
        else:
            index += 1
        if(len(index_arr) >= number):break
        if(len(index_arr) % 2000 == 0):print(len(index_arr))
        if(index + k >= shape):break 
    return index_arr,index_neighbor

train_data = pd.read_csv('./data/train/order_10/all_10_social1.csv')


index_arr,index_neighbor = sample_neg(train_data,number=60000)
# for index in tqdm(range(len(index_neighbor))):
#     index_neighbor[index] = index_neighbor[index][0]
# co_neigh = []
# for index in tqdm(range(len(index_neighbor))):
#     temp = index_neighbor[index]
#     co_neigh.append(temp)
data = train_data.iloc[index_arr]

data['history'] = data['history'].apply(lambda y :' '.join(remove_sep(eval(y))))
data['current'] = data['current'].apply(lambda y :' '.join(remove_sep(eval(y))))
# # # data['neighbor'] = co_neigh

np.save('./data/bert/15_10_con/social1/train_order.npy',data.values.tolist()[:50000])
np.save('./data/bert/15_10_con/social1/dev_order.npy',data.values.tolist()[50000:])
train_data = data

data = pd.read_csv('./data/test/order_10/all_10_social1.csv').iloc[:10000]
# co_neigh = []
# for index in tqdm(range(len(data))):
#     temp = neighbor[neighbor['user']==data.iloc[index]['id']]['nearst'][:k].values[0]
#     co_neigh.append(temp)
data['history'] = data['history'].apply(lambda y :' '.join(remove_sep(eval(y))))
data['current'] = data['current'].apply(lambda y :' '.join(remove_sep(eval(y))))
# data['neighbor'] = co_neigh
np.save('./data/bert/15_10_con/social1/test_order.npy',data.values.tolist())
test_data = data

2000
2000
4000
4000
6000
6000
8000
8000
10000
10000
12000
12000
14000
14000
16000
16000
18000
18000
20000
20000
22000
22000
24000
24000
26000
26000
28000
28000
30000
30000
30000
32000
32000
34000
34000
36000
36000
38000
38000
40000
40000
42000
42000
42000
44000
44000
46000
46000
48000
48000
50000
50000
52000
52000
54000
54000
56000
56000
58000
58000


/home/huangzhen/anaconda3/envs/news/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/huangzhen/anaconda3/envs/news/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
users_emb = pd.read_csv('./data/users_emb.csv')
emb_dict = dict()
for index ,row in tqdm(users_emb.iterrows()):
    emb_dict[row['user']] = row['emb']

# 7.数据处理

In [ ]:
# coding=utf-8
title_len = 10
train_title = pd.read_csv('./data/train/news.tsv',sep='\t',header=None, encoding='utf-8')
train_title.columns = ['id','category','sub_category','title','abstract','url','title_entity','abstract_entity']
test_title = train_title[['id','title']]
arr = []
for index ,row in train_title.iterrows():
    text = row['title'].split(' ')
    if(len(text) < title_len):
        text.extend(['[PAD]'] * (title_len - len(text)))
    else:
        text = text[:title_len]
    text.append('[SEP]')
    arr.append(text)
train_title['title'] = arr

test_title = pd.read_csv('./data/test/news.tsv',sep='\t',header=None, encoding='utf-8')
test_title.columns = ['id','category','sub_category','title','abstract','url','title_entity','abstract_entity']
test_title = test_title[['id','title']]
arr = []
for index ,row in test_title.iterrows():
    text = row['title'].split(' ')
    if(len(text) < title_len):
        text.extend(['[PAD]'] * (title_len - len(text)))
    else:
        text = text[:title_len]
    text.append('[SEP]')
    arr.append(text)
test_title['title'] = arr
title = pd.concat([train_title,test_title],0)

In [ ]:
title

In [ ]:

# train_behavior = pd.read_csv('./data/train/behaviors.tsv',sep ='\t',header=None)
# train_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
# test_behavior   = pd.read_csv('./data/test/behaviors.tsv',sep ='\t',header=None)
# test_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
# all_behavior = pd.concat([train_behavior,test_behavior],0)
# behavior_dict =  dict()
# for index,row in tqdm(all_behavior.iterrows()):
    
#     behavior_dict[row['id']] = row['clickHistory']
# train_id = train_data[['id']]
maxCount = 5
train_his = []
for index,row in tqdm(train_id.iterrows()):
    user = neighbor_dict[row['id']][2]##邻居用户
    history = behavior_dict[user]
    print(history)
    idxs = list(range(len(history)))
    np.random.shuffle(idxs)
    his_len = len(history)
    count = 0
    text = []
    for x in idxs:
        if(count == maxCount):break
        print(history[x])
        tit = title[title['id']==history[x]]['title'].values[0]
        if type(tit) == float:continue
        text.extend(tit)
        count += 1
    train_his.append(text)    
test_id = test_data[['id']]
test_his = []
for index,row in tqdm(test_id.iterrows()):
    user = neighbor_dict[row['id']]['clickHistory'] ##邻居用户
    history = behavior_dict[user]
    idxs = list(range(len(history)))
    np.random.shuffle(idxs)
    his_len = len(history)
    count = 0
    text = []
    for x in idxs:
        if(count == maxCount):break
        tit = title[title['id']==history[x]]['title'].values[0]
        if type(tit) == float:continue
        text.extend(tit)
        count += 1
    test_his.append(text) 

# 8.生成最终的数据

In [ ]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
from tqdm import tqdm
import numpy as np
import pandas as pd
train = np.load('./data/bert/15_10_con/train_order.npy',allow_pickle=True)
dev = np.load('./data/bert/15_10_con/dev_order.npy',allow_pickle=True)
test = np.load('./data/bert/15_10_con/test_order.npy',allow_pickle=True)
   
his_len = 150
curr_len = 10
def pad(text,his_len):
    arr = text.split(' ')
    if(len(arr) < his_len):
        arr.extend(['[PAD]'] * (his_len - len(arr)))
    else:
        arr = arr[:his_len]
    return ' '.join(arr)
train_file =[]
for row in tqdm(train):
    row[1] = pad(row[1],his_len)
    row[2] = pad(row[2],curr_len)
#     row[4] =  pad(row[4],neigh_len)
#     train_file.append(['[CLS] '+ row[1]+ ' ' + row[4] +  ' [SEP] ' + row[2] + ' [SEP]',row[3]])
    train_file.append(['[CLS] '+ row[1] +   ' [SEP] ' + row[2] + ' [SEP]',row[4]])
    
pd.DataFrame(train_file).to_csv('./data/bert/15_10_con/train_pair_order.csv',index = False)
dev_file = []
for row in tqdm(dev):
    row[1] = pad(row[1],his_len)
    row[2] = pad(row[2],curr_len)
# #     row[4] =  pad(row[4],neigh_len)
#     dev_file.append(['[CLS] '+ row[1]+ ' ' + row[4] +' [SEP] ' + row[2] + ' [SEP]',row[3]])
    dev_file.append(['[CLS] '+ row[1] +' [SEP] ' + row[2] + ' [SEP]',row[4]])
    
pd.DataFrame(dev_file).to_csv('./data/bert/15_10_con/dev_pair_order.csv',index = False)
test_file = []
for row in tqdm(test):
    row[1] = pad(row[1],his_len)
    row[2] = pad(row[2],curr_len)
#     row[4] =  pad(row[4],neigh_len)
#     test_file.append(['[CLS] '+ row[1]+ ' ' + row[4] +' [SEP] ' + row[2] + ' [SEP]',row[3]])
    test_file.append(['[CLS] '+ row[1] + ' [SEP] ' + row[2] + ' [SEP]',row[4]])
    
pd.DataFrame(test_file).to_csv('./data/bert/15_10_con/test_pair_order.csv',index = False)

In [ ]:
test = np.load('./data/bert/15_10_con/test_order.npy',allow_pickle=True)

In [ ]:
print(test[1][1])


In [ ]:
from keras_bert import extract_embeddings

train = np.load('./data/bert/15_10_con/train.npy')
dev = np.load('./data/bert/15_10_con/dev.npy')
test = np.load('./data/bert/15_10_con/test.npy')




In [ ]:
import pandas as pd
model_path = './data/outputs/bert_15_10_con'

from keras_bert import extract_embeddings, POOL_NSP, POOL_MAX
train_pair = pd.read_csv('./data/bert/15_10_con/train_pair.csv')
train_pair = train_pair['0'].apply(lambda x: eval(x)).values.tolist()
# dev_pair = pd.read_csv('./data/bert/15_10_con/dev_pair.csv')
# dev_pair = dev_pair['0'].apply(lambda x: eval(x)).values.tolist()
# test_pair = pd.read_csv('./data/bert/15_10_con/test_pair.csv')
# test_pair = test_pair['0'].apply(lambda x: eval(x)).values.tolist()
train_embed = []
for index,row in enumerate(train_pair):
    embed = extract_embeddings(model_path, row)[0][0]
    train_embed.append(embed)
    print(index)
    
# dev_embed = extract_embeddings(model_path, dev_pair)
# test_embed = extract_embeddings(model_path, test_pair)

In [ ]:
maxlen_history = 120
maxlen_curr = 30
import numpy as np
examples = []
step = 0
for index,row in all_train.iterrows():
    temp = []
    text = ' '.join(eval(row['history']))
    temp.append(text)
    text = ' '.join(eval(row['current']))
    temp.append(text)
    temp.append(row['label'])
    examples.append(temp)
    step += 1
    if(step % 10000 == 0):print(step)


In [ ]:

print( maxlen_history)
print(maxlen_curr)
### 训练数据裁剪
k = 1
train_1 = []
index = 0
lens = len(examples)
while index <(lens):
    flag = True
    for his in range(k):
        if examples[index+his][2] != 0:
            flag = False
            break
    if(flag):
        for pos in range(k):
            history = examples[pos + index]
            history[0] = history[0]
            train_1.append(history)
        index += k
        if(index + k >= lens):break
        while(examples[index][2] != 1):
            index += 1
            if(index + k >= lens):break
        if(index + k >= len(examples)):break
        history = examples[index]
        train_1.append(history)
    else:
        index += 1
    if(len(train_1) % 2000 == 0):print(len(train_1))
    if(index + k >= lens):break 

In [ ]:

texts = [('all work and no play', 'makes jack a dull boy'),
    ('makes jack a dull boy', 'all work and no play')]
embeddings = extract_embeddings(model_path, texts)[0][0]

In [ ]:
len(embeddings)

In [ ]:
embeddings[1][7]

In [ ]:
import torch
from transformers import BertTokenizer

model_name = './data/outputs/bert_15_10_con'  # 指定需下载的预训练模型参数

# BERT 在预训练中引入了 [CLS] 和 [SEP] 标记句子的开头和结尾
samples = ['[CLS] are you ok？ 0 [SEP]I amgo fine ! 0 [SEP]']  # 准备输入模型的语句

tokenizer = BertTokenizer.from_pretrained(model_name)

tokenized_text = [tokenizer.tokenize(i) for i in samples]

input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
input_ids = torch.LongTensor(input_ids)
print(input_ids)

In [ ]:
import pandas as pd
data = pd.read_csv('./data/train/seprate_10/all_10.csv')

In [ ]:
data